In [28]:
import os
import csv

# Directory containing your ".out" files
directory = '.'
output_csv = 'integral_data.csv'
threshold = 0.00000001  # Set the threshold here

# Initialize dictionaries to hold data
one_electron_data = {}
two_electron_data = {}
x_values = set()

# Function to parse files
def parse_files():
    for filename in os.listdir(directory):
        if filename.endswith(".out"):
            x_value = float(filename.replace(".out", ""))
            x_values.add(x_value)
            filepath = os.path.join(directory, filename)

            with open(filepath, 'r') as file:
                content = file.readlines()

            inside_one_electron = False
            inside_two_electron = False

            for line in content:
                line = line.strip()
                
                if line.startswith("begin_one_electron_integrals"):
                    inside_one_electron = True
                    continue
                elif line.startswith("end_one_electron_integrals"):
                    inside_one_electron = False
                    continue
                
                if line.startswith("begin_two_electron_integrals"):
                    inside_two_electron = True
                    continue
                elif line.startswith("end_two_electron_integrals"):
                    inside_two_electron = False
                    continue

                # Parse the values inside each integral block
                if inside_one_electron:
                    parts = line.split()
                    if len(parts) == 3:
                        label = (int(parts[0]), int(parts[1]))
                        value = float(parts[2])
                        if label not in one_electron_data:
                            one_electron_data[label] = {}
                        one_electron_data[label][x_value] = value

                elif inside_two_electron:
                    parts = line.split()
                    if len(parts) == 5:
                        label = (int(parts[0]), int(parts[1]), int(parts[2]), int(parts[3]))
                        value = float(parts[4])
                        if label not in two_electron_data:
                            two_electron_data[label] = {}
                        two_electron_data[label][x_value] = value

# Run parsing function
parse_files()

# Sort x_values for consistent column ordering
x_values = sorted(x_values)

# Write data to CSV with threshold filtering
with open(output_csv, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)

    # Write header: Label columns and X values as subsequent columns
    header = ["Label"] + [f"{x:.2f}" for x in x_values]
    csv_writer.writerow(header)

    # Function to check if all values in a row meet the threshold
    def meets_threshold(data, x_vals):
        values = [data.get(x, float('-inf')) for x in x_vals]
        return all(abs(val) >= threshold for val in values)

    # Write one-electron data with threshold filtering
    for label, data in one_electron_data.items():
        if meets_threshold(data, x_values):
            row = [label] + [data.get(x, "") for x in x_values]
            csv_writer.writerow(row)

    # Write two-electron data with threshold filtering
    for label, data in two_electron_data.items():
        if meets_threshold(data, x_values):
            row = [label] + [data.get(x, "") for x in x_values]
            csv_writer.writerow(row)

print(f"Data with values above threshold {threshold} has been extracted to {output_csv} in table format.")


Data with values above threshold 1e-08 has been extracted to integral_data.csv in table format.


In [13]:
import csv
import matplotlib.pyplot as plt
from ipywidgets import interact, Dropdown
import numpy as np

# CSV file containing the processed data
input_csv = 'integral_data.csv'

# Initialize dictionary to hold data from CSV
data = {}
x_values = []

# Load data from CSV
with open(input_csv, 'r') as csvfile:
    csv_reader = csv.reader(csvfile)
    header = next(csv_reader)  # First row is the header

    # Extract x values from header
    x_values = [float(label) for label in header[1:]]

    # Load each label's data
    for row in csv_reader:
        label = eval(row[0])  # Convert string tuple back to tuple
        y_values = [float(value) if value else np.nan for value in row[1:]]
        data[label] = y_values

# Function to plot selected data
def plot_data(label):
    y_vals = data.get(label)
    if y_vals is None:
        print(f"No data found for label {label}")
        return

    plt.figure(figsize=(10, 6))
    plt.plot(x_values, y_vals, marker='o', linestyle='-')
    plt.xlabel('X Value')
    plt.ylabel('Integral Value')
    plt.title(f'Plot for Label {label}')
    plt.grid(True)
    plt.show()

# Create dropdown options
dropdown = Dropdown(options=list(data.keys()), description="Select Label:")

# Create interactive plot
interact(plot_data, label=dropdown)


interactive(children=(Dropdown(description='Select Label:', options=((1, 1), (3, 1), (5, 1), (2, 2), (4, 2), (…

<function __main__.plot_data(label)>